In [4]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

# mnist

In [55]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [56]:
device

'cuda'

In [57]:
learning_rate=0.001
training_epoch=15
batch_size=100

In [58]:
mnist_train= dsets.MNIST(root='MNIST_data/',
                        train=True, transform=transforms.ToTensor(),
                        download=True)
mnist_test= dsets.MNIST(root='MNIST_data/',
                        train=False, transform=transforms.ToTensor(),
                        download=True)

In [59]:
data_loader= torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,
                                        shuffle=True,drop_last=True)

In [60]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1= nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer2= nn.Sequential(
            nn.Conv2d(32,64,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.fc=nn.Linear(7*7*64, 10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight) #초기화
        
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0),-1)
        out=self.fc(out)
        return out
        
        
        

In [61]:
model= CNN().to(device)

In [62]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [63]:
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [23]:
#train
total_batch=len(data_loader)
print(total_batch)
for epoch in range(training_epoch):
    avg_cost=0
    
    for X,Y in data_loader:
        X= X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis=model(X)
        cost=criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        avg_cost+= cost/ total_batch
    print('Epoch:{} cost= {}'.format(epoch+1,avg_cost))


600
Epoch:1 cost= 0.006377627607434988
Epoch:2 cost= 0.00410427013412118
Epoch:3 cost= 0.004087864421308041
Epoch:4 cost= 0.003787226742133498
Epoch:5 cost= 0.004933951888233423
Epoch:6 cost= 0.004547517746686935
Epoch:7 cost= 0.002257126849144697
Epoch:8 cost= 0.0024167962837964296
Epoch:9 cost= 0.00363301788456738
Epoch:10 cost= 0.002311863237991929
Epoch:11 cost= 0.00346028502099216
Epoch:12 cost= 0.0028967224061489105
Epoch:13 cost= 0.0037777230609208345
Epoch:14 cost= 0.001077996683306992
Epoch:15 cost= 0.0002023927663685754


In [25]:
with torch.no_grad():
    X_test=mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test=mnist_test.test_labels.to(device)
    
    prediction=model(X_test)
    correct_prediction=torch.argmax(prediction,1)==Y_test
    accuracy=correct_prediction.float().mean()
    print(accuracy.item())

C:\Users\mjsk1\anaconda3\envs\Pytorch\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\mjsk1\anaconda3\envs\Pytorch\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


0.9887999892234802


# visdom

In [5]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets

In [8]:
import visdom
vis= visdom.Visdom()

Setting up a new session...


In [10]:
vis.text('hello',env='main')

'window_3c8cebcb0c7fa8'

In [11]:
a=torch.randn(3,200,200)
vis.image(a)

'window_3c8cebe91e818a'

In [13]:
vis.images(torch.Tensor(3,3,28,28))

'window_3c8cec187f30f0'

In [16]:
mnist= dsets.MNIST(root="./MNIST_data",train=True,transform=torchvision.transforms.ToTensor()
                  ,download=True)

In [18]:
cifar10= dsets.CIFAR10(root="./MNIST_data",train=True,transform=torchvision.transforms.ToTensor()
                  ,download=True)

100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [00:35<00:00, 4822632.39it/s]


Extracting ./MNIST_data\cifar-10-python.tar.gz to ./MNIST_data


In [21]:
data= cifar10.__getitem__(0)
print(data[0].shape)

torch.Size([3, 32, 32])


In [22]:
vis.images(data[0],env='main')

'window_3c8cecbd56ae1e'

In [23]:
data_loader= torch.utils.data.DataLoader(dataset=mnist,batch_size=32,shuffle=False)

In [27]:
enumerate(data_loader)

In [24]:
for num, value in enumerate(data_loader):
    value=value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [54]:
vis.close(env='main') # 창 끄기

''

In [37]:
Y_data=torch.randn(5)
plt=vis.line(Y=Y_data)

In [38]:
X_data=torch.Tensor([1,2,3,4,5])
plt=vis.line(Y=Y_data,X=X_data)

In [39]:
Y_append=torch.randn(1)
X_append=torch.Tensor([6])
vis.line(Y=Y_append,X=X_append,win=plt,update='append')

'window_3c8cee68735cb8'

In [44]:
num= torch.Tensor(list(range(0,10)))
num=num.view(-1,1)
num=torch.cat((num,num),dim=-1)
num
plt=vis.line(Y=torch.randn(10,2),X=num)

In [47]:
plt=vis.line(Y=Y_data,X=X_data,opts=dict(title='test',showlegend=True))

In [48]:
plt=vis.line(Y=Y_data,X=X_data,opts=dict(title='test',legend=['1번'],showlegend=True))

In [50]:
plt=vis.line(Y=torch.randn(10,2),X=num,opts=dict(title='test',legend=['1번','2번'],showlegend=True))

In [64]:
def loss_tracker(loss_plot,loss_value,num):
    vis.line(X=num,Y=loss_value,win=loss_plot,update='append')

In [53]:
plt=vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss=torch.randn(1)+i
    loss_tracker(plt,loss,torch.Tensor([i]))

# mnist-visdom

In [67]:
vis.close(env='main') 

''

In [68]:
loss_plt= vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker',legend=['loss'],showlegend=True))

In [69]:
#train
total_batch=len(data_loader)
print(total_batch)
for epoch in range(training_epoch):
    avg_cost=0
    
    for X,Y in data_loader:
        X= X.to(device)
        Y=Y.to(device)
        
        optimizer.zero_grad()
        hypothesis=model(X)
        cost=criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        avg_cost+= cost/ total_batch
    print('Epoch:{} cost= {}'.format(epoch+1,avg_cost))
    loss_tracker(loss_plt,torch.Tensor([avg_cost]),torch.Tensor([epoch]))

600
Epoch:1 cost= 0.22556628286838531
Epoch:2 cost= 0.06302006542682648
Epoch:3 cost= 0.04633789882063866
Epoch:4 cost= 0.037422701716423035
Epoch:5 cost= 0.03146076947450638
Epoch:6 cost= 0.026114581152796745
Epoch:7 cost= 0.022100256755948067
Epoch:8 cost= 0.018162861466407776
Epoch:9 cost= 0.016209902241826057
Epoch:10 cost= 0.013248170726001263
Epoch:11 cost= 0.010320274159312248
Epoch:12 cost= 0.009862552396953106
Epoch:13 cost= 0.008333929814398289
Epoch:14 cost= 0.0061579737812280655
Epoch:15 cost= 0.007398598827421665
